# Neural Networks in Keras

Personal notes on Udacity MLND Deep Learning section.

Let's start with a singel hidden layer model:


In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation


# X shape: (num_rows, num_cols)
# training data stored as row vectors
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)

# y must have an output vector for each input vector
y = np.array([[0], [0], [0], [1]], dtype=np.float32)


'''
Create sequential model.
This is a wraper to treat the network as a sequence of layers that implements the model interface with
methods `compile()`, `fit()`, `evaluate()`... used to train and run the model.
'''
model = Sequential()

'''
The Layer class provides common interface for a variety of layers (fully connected, max pool, activation, ...)
You can add a layer to a model using the model's `add()` method
'''
# 1st Layer - Add an input layer of 32 nodes with the same input shape as the training samples in X
# you only have to explicitly set the input dimensions for the first layer
model.add(Dense(32, input_dim=X.shape[1]))

# Add a softmax activation layer
model.add(Activation('softmax'))

# 2nd Layer - Add a fully connected output layer (1 node)
model.add(Dense(1))

# Add a sigmoid activation layer
model.add(Activation('sigmoid'))

'''
Compiling
Call backend (TF), and bind loss function, optimizer, evaluation metric, ...
'''
#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])

'''
See the resulting model architecture
'''
model.summary()

'''
Train the model
'''
model.fit(X, y, epochs=1000, verbose=0)

# Stochastic Gradient Descent, specify batches:
##model.fit(X_train, y_train, epochs=1000, batch_size=100, verbose=0)

'''
Evaluate the model
'''
model.evaluate(X, y) # score = model.evaluate(x_test, y_test, verbose=0) 
                     # https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                96        
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 9ms/step


[0.28675633668899536, 1.0]

## xor quiz

Code used in the interface:

```
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
# Using TensorFlow 1.0.0; use tf.python_io in later versions
tf.python.control_flow_ops = tf

# Set random seed
np.random.seed(42)

# Our data
X = np.array([[0,0],[0,1],[1,0],[1,1]]).astype('float32')
y = np.array([[0],[1],[1],[0]]).astype('float32')

# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation

# Building the model
xor = Sequential()

# Add required layers
xor.add(Dense(8, input_dim=2))
xor.add(Activation('tanh'))
xor.add(Dense(1))
xor.add(Activation('sigmoid'))

# Specify loss as "binary_crossentropy", optimizer as "adam",
# and add the accuracy metric
xor.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])

# Uncomment this line to print the model architecture
xor.summary()

# Fitting the model
history = xor.fit(X, y, nb_epoch=300, verbose=0)

# Scoring the model
score = xor.evaluate(X, y)
print("\nAccuracy: ", score[-1])

# Checking the predictions
print("\nPredictions:")
print(xor.predict_proba(X))
```

Output:

```
Using TensorFlow backend.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
====================================================================================================
dense_1 (Dense)                  (None, 8)             24          dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 8)             0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             9           activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 1)             0           dense_2[0][0]                    
====================================================================================================
Total params: 33
Trainable params: 33
Non-trainable params: 0
____________________________________________________________________________________________________

4/4 [==============================] - 0s

Accuracy:  1.0

Predictions:

4/4 [==============================] - 0s
[[ 0.42648906]
 [ 0.51158553]
 [ 0.57282132]
 [ 0.48525804]]
```

To split in test and training sets:

```
# Test and training sets example

(X_train, X_test) = X[50:], X[:50]
(y_train, y_test) = y[50:], y[:50]
```

## Overfiting Management

Underfitting => High bias
Overfitting => High variance

Go for overfitting and then try to reduce it:

### 1. Stop early  
Right number of epochs: plot epochs vs training / testing error [Model Complexity Graph]

### 2. Regularization
Large coefficients tend to overfit.


In [4]:
# Large coefficient example

'''
Separate (1,1) and (-1,-1)
first model: x_1 + x_2 => w_1=1, w_2=1
second model: 10x_1 + 10x_2 => w_1=10, w_2=10
'''

# apply sigmoid function
from scipy.special import expit

# Model 1
print('Model 1 first prediction:', expit(1+1)) # (1,1) prediction
print('Model 1 second prediction:', expit(-1-1)) # (-1,-1) prediction

# Model 2
print('Model 2 first prediction:', expit(10+10)) # (1,1) prediction
print('Model 2 second prediction:', expit(-10-10)) # (-1,-1) prediction

Model 1 first prediction: 0.880797077978
Model 1 second prediction: 0.119202922022
Model 2 first prediction: 0.999999997939
Model 2 second prediction: 2.06115361819e-09


The second model is overffiting.

To penalize large weights one can add a term in the error function that is the sums of the absolute values of the weights [L1 Regularization], or the sum of the squares of the weights [L2 Regularization], in any case times a constant lambda. 

L1 yields sparse vectors (small vectors go to 0). Good for feature selection.

L2 tries to mantain all the weights homogeneously small. Normally better for training models.

### 3. Dropout

As training goes through epochs, randomly turn off some of the nodes. To do this we give the algorithm a new parameter for the nodes. This parameter is the probability that a node gets dropped at a given epoch.

Applying dropout in Keras:

In [7]:
'''
Ref: https://keras.io/getting-started/sequential-model-guide/#examples
'''
import keras
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model2 = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model2.add(Dense(64, activation='relu', input_dim=20))
model2.add(Dropout(0.2)) # After adding layers, add dropout
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2)) # In each epoch, each node gets dropped off with a probability of 0.2
model2.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model2.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model2.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 1s 502us/step - loss: 2.3505 - acc: 0.0920
Epoch 2/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3112 - acc: 0.1180
Epoch 3/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.3259 - acc: 0.1020
Epoch 4/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.3024 - acc: 0.1090
Epoch 5/20
1000/1000 [==============================] - 0s 25us/step - loss: 2.3019 - acc: 0.1180
Epoch 6/20
1000/1000 [==============================] - 0s 25us/step - loss: 2.3040 - acc: 0.1130
Epoch 7/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3014 - acc: 0.1190
Epoch 8/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.3045 - acc: 0.1110
Epoch 9/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.2957 - acc: 0.1320
Epoch 10/20
1000/1000 [==============================] - 0s 25us/step - loss: 2.3014 - acc: 0.1070
Epoch 11/20
1000/1

## Vanishing Gradient

The sigmoid function gets flat at both ends, and its derivatives tend to zero on both directions. using the chain rule this tendency exacerbates. Leave output as sigmoid (for probability), and change activation functions in other layers.

Alternative activation functions (Keras):

```
model.add(Activation('relu'))
model.add(Activation('tanh'))
```

## Local Minima

1. Use random restart.
2. Use momentum [beta], a constant from 0 to 1: the average of last weighted steps (last step matters most):
step(n) + beta * step(n-1) + beta^2 * step(n-2) + ...


## Keras Optimizers

Ref:

[Optimizers - Keras Docs](https://keras.io/optimizers/)

[Gradient Descent optimization algos](http://ruder.io/optimizing-gradient-descent/index.html#rmsprop)

### Stochastic Gradient Descent (SGD)
Parameters: learning rate, momentum, nesterov momentum (slows down gradient when close to solution)

### Adam
Adaptive Moment Estimation. Uses exponential decay that consists of the average (first moment), and the variance (second moment) of the previous steps.

### RMSProp
Root Mean Squared Error. Decreases the learning rate by dividing it by an exponentially decaying average of squared gradients.